In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('mlpractise').getOrCreate()

In [ ]:
spark

In [ ]:
train_df = spark.read.csv('/content/drive/MyDrive/Data/emp-data-mlib.csv',header=True,inferSchema=True)

In [ ]:
train_df.show()

+---------+---+----------+------+
|     name|age|experience|salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [ ]:
train_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [ ]:
train_df.columns

['name', 'age', 'experience', 'salary']

**Vector Assembler**

In [ ]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['age', 'experience'], outputCol='Independent Features')

In [ ]:
output = feature_assembler.transform(train_df)

In [ ]:
output.show()

+---------+---+----------+------+--------------------+
|     name|age|experience|salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [ ]:
output.columns

['name', 'age', 'experience', 'salary', 'Independent Features']

In [ ]:
final_data = output.select('Independent Features','salary')

In [ ]:
final_data.show()

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



**randomSplit**

In [ ]:
from pyspark.ml.regression import LinearRegression
train_data , test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features',labelCol='salary')
regressor = regressor.fit(train_data)

In [ ]:
regressor.coefficients

DenseVector([-58.8235, 1666.6667])

In [ ]:
regressor.intercept

15196.078431372438

In [ ]:
pred_results = regressor.evaluate(test_data)

In [ ]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18784.313725490196|
|          [30.0,8.0]| 25000|26764.705882352937|
+--------------------+------+------------------+



In [ ]:
pred_results.meanAbsoluteError ,pred_results.meanSquaredError

(1490.1960784313706, 2296039.9846212924)